In [1]:
import requests
import numpy as np
import pandas as pd
import os
import time
import datetime
import re
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time
import numpy as np
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
#讀取檔案
col_dict = {"統編": str, "名稱": str, "備註": str}
corp_file = pd.read_excel('C:/Users/user/Downloads/corp_search_total_done.xlsx', dtype=col_dict)
#testing case
corp_name_total = list(corp_file['名稱'])

In [3]:
#過濾網站名單
black_list = [
    'www.104.com.tw',
    'www.1111.com.tw',
    'www.taiwanslot.com.tw',
    'www.yes123.com.tw',
    'www.twincn.com',
    'www.findcompany.com.tw',
    'alltwcompany.com',
    'alltwbiz.com',
    'www.twfile.com',
    'twcorp.tw',
    'twypage.com',
    'opengovtw.com',
    'gotw101.com',
    'twinc.com.tw',
    'costring.com',
    'teach.datagove.com',
    'www.1111tc.com.tw',
    'interview.tw',
    'datagovtw.com',
    'tiponet.tipo.gov.tw',
    'www.iyp.com.tw',
    'm.iyp.com.tw',
    'trade.1111.com.tw',
    'ns.com.tw',
    'www.518.com.tw',
    'salary.tw',
    'poi.zhupiter.com',
    'web66.com.tw',
    'www.tmbia.org.tw',
    'mo.com.tw',
    'smb.nss.com.tw',
    'www.taiwanbuying.com.tw',
    'abiz.tw',
    'aibee.com.tw',
    'www.companys.com.tw',
    '121.com.tw',
    'tw.stock.yahoo.com',
    'hragency.imobile01.com',
    'abiz.tw',
    'tw.datagove.com',
    'company.5432.tw',
    'fda188.datagove.com',
    'data.zhupiter.com',
    'mo.com.tw',
    'hc.store.tnn.tw',
    '121.com.tw',
    'findcorp.tw',
    'tw.bizpo.net',
    'opendatatw.com',
    'www.moc.gov.tw',
    'notebz.com',
    'www.ftc.gov.tw',
    'tw66.com.tw',
    'find168.com.tw',
    'www.arch-world.com.tw',
    'tw.indeed.com',
    'fda36.datagove.com',
    'zh-tw.facebook.com',
    'www.facebook.com',
    'aibee.tw',
    'job.taiwanjobs.gov.tw',
    'biz.5168.mx',
    'huangyedu.com',
    'www.ezsogosi.com',
    'tw.ypgo.net',
    'www.web393.com',
    'fda68.datagove.com',
    'www.tuugo.tw',
    'data.bznk.com',
    'taiwan.datagove.com',
    'hotel.imobile01.com',
    'smartpark.tw',
    'vip.arch-world.com.tw',
    'tw.gothejob.com',
    'www.bizzword.com',
    'ofdays.com',
    'twnypage.com'
]

In [12]:
# corp_name = corp_name_total[121:150]
corp_name = list(corp_file[(corp_file['負責同仁']=='')&(corp_file['raw data'].isna())]['名稱'])[50:50]

In [13]:
len(corp_name)

50

In [6]:
def open_browser():
    #selenium settings
    chrome_options = webdriver.ChromeOptions()
    #無痕模式
    chrome_options.add_argument("--incognito")
    #將擴充套件放入至Webdriver的開啟網頁內容
    # chrome_options.add_extension('./7.0.10_0.crx')
    # chrome_options.add_argument('--proxy-server=%s' % proxy_ip)
    chrome = webdriver.Chrome('./chromedriver',options=chrome_options)
    #設定等待時間
    chrome.implicitly_wait(5)
    chrome.set_page_load_timeout(15)
    return chrome

In [40]:
# # i am not robot check
# # find iframe
# captcha_iframe = WebDriverWait(chrome, 10).until(
#     ec.presence_of_element_located(
#         (
#             By.TAG_NAME, 'iframe'
#         )
#     )
# )

# ActionChains(chrome).move_to_element(captcha_iframe).click().perform()

# # click im not robot
# captcha_box = WebDriverWait(chrome, 10).until(
#     ec.presence_of_element_located(
#         (
#             By.ID, 'g-recaptcha-response'
#         )
#     )
# )

# chrome.execute_script("arguments[0].click()", captcha_box)

In [77]:
# #開啟VPN
# chrome.get("chrome-extension://gjknjjomckknofjidppipffbpoekiipm/panel/index.html")
# #透過find_element_by_xpath找到點擊的位置並且點擊
# # chrome.find_element(By.XPATH,'//*[@id="screenMain"]/div[3]/button[2]').click()
# chrome.find_element(By.XPATH,'//*[@id="screenMain"]/div[3]/button[1]').click()

In [14]:
#loop
n_pages = 3
counting_done = 1
# chrome = open_browser()
for corp in corp_name:
    if (counting_done//5)%2==1:
        chrome.get('https://www.google.com.tw/')
    elif (counting_done//5)%2==0:
        chrome.get('https://www.google.com/')
    #search bar
    search = chrome.find_element("name","q")
    search.send_keys(corp)
    time.sleep(np.random.randint(5,7))
    search.send_keys(Keys.RETURN)
    try:
        #判斷是否進入機器人頁面
        chrome.find_element("link text","下一頁")
        #擷取搜尋結果頁面
        names = []
        links = []
        counter = 0
        while counter < n_pages:
            time.sleep(np.random.randint(8,15))
            soup = BeautifulSoup(chrome.page_source, 'html.parser')
            search = soup.find_all('div', class_="yuRUbf")
            for h in search:
                #website name
                names.append(h.h3.text)
                #website link
                links.append(h.a.get('href'))
            counter = counter+1
            try:
                #點擊下一頁
                chrome.find_element("link text","下一頁").click()
            except:
                print("第{}筆,名稱為「{}」,搜尋結果不足3頁".format(corp_name_total.index(corp),corp))
                break
        time.sleep(np.random.randint(5))
        #原始所有結果
        tuple_list = list(zip(names, links))
        #原始結果附於表格內
        corp_file.loc[corp_file["名稱"]==corp,"raw data"] = str(tuple_list)
        #過濾網頁
        for i in list(tuple_list):
            if any(substring in i[1] for substring in black_list):
                tuple_list.remove(i)
        counting_done = counting_done+1
        #過濾結果前5(不足5筆error跳去except)
        corp_file.loc[corp_file["名稱"]==corp,"網站名稱1"] = tuple_list[0][0]
        corp_file.loc[corp_file["名稱"]==corp,"網址1"] = tuple_list[0][1]
        corp_file.loc[corp_file["名稱"]==corp,"網站名稱2"] = tuple_list[1][0]
        corp_file.loc[corp_file["名稱"]==corp,"網址2"] = tuple_list[1][1]
        corp_file.loc[corp_file["名稱"]==corp,"網站名稱3"] = tuple_list[2][0]
        corp_file.loc[corp_file["名稱"]==corp,"網址3"] = tuple_list[2][1]
        corp_file.loc[corp_file["名稱"]==corp,"網站名稱4"] = tuple_list[3][0]
        corp_file.loc[corp_file["名稱"]==corp,"網址4"] = tuple_list[3][1]
        corp_file.loc[corp_file["名稱"]==corp,"網站名稱5"] = tuple_list[4][0]
        corp_file.loc[corp_file["名稱"]==corp,"網址5"] = tuple_list[4][1]
        #定期存檔紀錄
        if counting_done%25==0:
            print("已完成{}筆".format(counting_done))
            corp_file.to_excel('C:/Users/user/Downloads/corp_search_total_done.xlsx',index=False)
        #50筆重啟一次
        if counting_done%50==0:
            chrome.close()
            chrome = open_browser()
    except:
        counting_done = counting_done+1
        print('I am not a robot, 需要人工點選/符合條件家數不足5筆')
        print("出現異常需檢查,目前資料處理至第{}筆,名稱為「{}」".format(corp_name_total.index(corp),corp))
        time.sleep(np.random.randint(5))

I am not a robot, 需要人工點選/符合條件家數不足5筆
出現異常需檢查,目前資料處理至第3540筆,名稱為「良泰機車行」
已完成25筆


In [8]:
corp_file.to_excel('C:/Users/user/Downloads/corp_search_total_done.xlsx',index=False)